In [8]:
!pip install -q neo4j wikipedia lxml gpt4all
#!pip -q install -U pip setuptools wheel
#!pip install jupyter_contrib_nbextensions
!pip install setuptools==58.2.0
!pip install --upgrade --no-deps --force-reinstall -q crosslingual_coreference 
!pip install -U -q --use-pep517 allennlp nltk 'spacy[cuda12x,lookups]'
!python -m -q spacy download en_core_web_sm

  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [270 lines of output]
        Using cached setuptools-69.1.1-py3-none-any.whl.metadata (6.2 kB)
        Using cached wheel-0.32.3-py2.py3-none-any.whl (21 kB)
        Using cached Cython-3.0.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.2 kB)
        Using cached cymem-2.0.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.4 kB)
        Using cached preshed-2.0.1.tar.gz (113 kB)
        Preparing metadata (setup.py): started
        Preparing metadata (setup.py): finished with status 'done'
        Using cached murmurhash-1.0.10-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.0 kB)
        Using cached thinc-7.0.8.tar.gz (1.9 MB)
        Preparing metadata (setup.py): started
        Preparing metadata (setup.py): finished with status 'done

In [4]:
import allennlp

ModuleNotFoundError: No module named 'allennlp'

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

NEO_DB = os.getenv('NEO_DB')
NEO_USER = os.getenv('NEO_USER')
NEO_PASS = os.getenv('NEO_PASS')



In [ ]:
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Neo4jVector
from langchain_community.embeddings import GPT4AllEmbeddings

# Load documents

In [ ]:


# Read the wikipedia article
raw_documents = WikipediaLoader(query="The Witcher", load_max_docs=3).load()

# Define chunking strategy
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=20
)
# Chunk the document
documents = text_splitter.split_documents(raw_documents)
# Remove the summary
for d in documents:
    del d.metadata["summary"]



In [ ]:
embeddings = GPT4AllEmbeddings()

In [ ]:
url="neo4j://192.168.68.84:7687/"

neo4j_db = Neo4jVector.from_documents(
    documents,
    embeddings,
    url=url,
    username=NEO_USER,
    password= NEO_PASS,
    #database=NEO_DB,  # neo4j by default
    index_name="wikipedia",  # vector by default
    node_label="WikipediaArticle",  # Chunk by default
    text_node_property="info",  # text by default
    embedding_node_property="vector",  # embedding by default
    create_id_index=True,  # True by default
)

In [ ]:
existing_index = Neo4jVector.from_existing_index(
    embedding=embeddings,
    url=url,
    username=NEO_USER,
    password=NEO_PASS,
    index_name="wikipedia",
    text_node_property="info",  # Need to define if it is not default
)

In [ ]:
print(existing_index.node_label) # WikipediaArticle
print(existing_index.embedding_node_property) # vector

In [ ]:
query = "Who was the witcher in the video game?"
docs_with_score = existing_index.similarity_search_with_score(query, k=3)

In [ ]:
len(docs_with_score)

In [ ]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content[:200])
    print("-" * 80)

# Information Extraction 

In [ ]:
# GET DOCS

# CHUNK 

# LLM TO CLEAN 

# LLM TO FORMAT 

In [ ]:
len(raw_documents)

In [ ]:

import spacy
import crosslingual_coreference

DEVICE = 0 # Number of the GPU, -1 if want to use CPU

# Add coreference resolution model
coref = spacy.load('en_core_web_sm', disable=['ner', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer'])
coref.add_pipe("xx_coref", config={"chunk_size": 2500, "chunk_overlap": 2, "device": DEVICE})